# Homework 5: Logistic Regression and Support Vector Machines

by Natalia Frumkin and Karanraj Chauhan with help from B. Kulis, R. Manzelli, and A. Tsiligkardis

## Problem 1: SVM Toy Example

Given the following two-class data set:

**Class -1: **
A = (1,1)
B = (2,3)

**Class +1: **
C = (2,5)
D = (4,2)

<ol type="a">
  <li>Plot the data.</li>
  <li>Plot the hyperplane described by w = $(3,2)^T, b = -12$</li>
  <li>Calculate the $l_2$ distance of data point C from the hyperplane.</li>
  <li>Determine if the hyperplane linearly separates the data. Explain.</li>
  <li>Calculate the hard margin SVM hyperplane in canonical form.</li>
  <li>Which, if any, data points lie on the SVM hyperplane?</li>
</ol>

## Problem 2: Logistic Regression

<p>In this problem, we will use a logistic regression model to classify emails as "spam" (1) or "non-spam" (0). Recall that the hypothesis/decision rule in a logistic regression model is given by</p>

$$h_\theta(x) = \sigma(\theta^Tx) \\ \text{where } \sigma  \text{ is the sigmoid function}$$

<p>Since logistic regression does not have a closed form solution, we will use gradient descent to obtain the parameters $\theta$. We will use the negative log likelihood loss with L2 regularization as the loss function. Mathematically, the loss function $l(\theta)$ for a given set of parameters $\theta$ will be,</p>

$$l(\theta) = NLL(\theta) + \frac{\lambda}{2}||\theta||^2 \\ \text{where } NLL(\theta) = -\sum_{i=1}^{n} y_i\log(h(x_i)) + (1 - y_i)\log(1 - h(x_i))$$

<p>The good news is, you won't have to worry about these equations for implementing gradient descent (hurray!). However, what you will need is the gradient or the derivative of the loss function. For a given $n$$ x $$d$ matrix $X$ of data, $n$ x $1$ vector of labels (0/1) $y$, and corresponding $n$ x $1$ vector of predictions $\hat{y}$, the loss function gradient is</p>

$$\nabla l(\theta) = (\hat{y} - y)^{T} \cdot X + \lambda \cdot \theta$$

<ol type="a">
    <li>Load the dataset file spambase_data.csv using pandas, and then split the dataset into a train set and a test set. Note: train/test ratio of 0.8/0.2 has been known to work, but you are welcome to try other values.</li>
    <li>Using the loss gradient equation above, implement gradient descent (use only the train set for this) to find the parameters $\theta$ of the logistic regression model. Note: $learning$ $rate = 0.00001$, $\lambda$ = $10$, and $number$ $of$ $steps = 3000$ have been known to give a decent accuracy but you are welcome to try other values, especially for $number$ $of$ $steps$.</li>
    <li>Report the correct classification rate (CCR) of the model on train data and test data. The CCR is defined as $$CCR = \frac{num\_correct\_predictions}{num\_samples}$$</li>   
</ol>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

import pdb

In [11]:
# read in raw dataset
df = pd.read_csv('spambase_data.csv')
df.head()    # sanity check

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [5]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2)

# convert pandas dataframe/series to numpy objects
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [6]:
# append column of 1's to X_train to account for bias
X_train = np.hstack([X_train, np.ones(shape=(X_train.shape[0], 1))])
X_test = np.hstack([X_test, np.ones(shape=(X_test.shape[0], 1))])

# reshape y's
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [10]:
# utility functions - these are functions that will come in handy while coding the main part
def sigmoid(x):
    """Computes sigmoid of elements of x

    Does computation in a more numerically stable way by taking care of edge cases
    i.e. x = inf and x = -inf

    This implementation throws RuntimeWarnings. Use mysigmoid to avoid them.
    """
    return np.where(x >= 0,
                   1 / (1 + np.exp(-x)),
                   np.exp(x) / (1 + np.exp(x)))

def mysigmoid(x):
    """Computes sigmoid of elements of x

    Does computation in a more numerically stable way by taking care of edge cases
    i.e. x = inf and x = -inf

    The above implementation throws RuntimeWarnings. This does not.
    """
    s = np.empty_like(x)
    for i in range(x.shape[0]):
        xi = x[i, 0]
        if xi > 0:
            s[i, 0] = 1 / (1 + np.exp(-1*xi))
        else:
            s[i, 0] = np.exp(xi) / (1 + np.exp(xi))

    return s


def sigmoid_derivative(x):
    """Computes derivative of sigmoid of x
    Uses the property : sigmoid derivative = sigmoid * (1 - sigmoid)
    """
    s = mysigmoid(x)
    return s * (1 - s)


def nll_loss(y_true, y_pred):
    """Negative log likelihood function

    Arguments:
        y_true {ndarray} -- (num_elements, 1) shaped ndarray of true (binary) labels
        y_pred {ndarray} -- (num_elements, 1) shaped ndarray of predicted labels (continuous values in [0, 1])

    Returns:
        [type] -- [description]
    """
    return np.where(y_true == 1,
                   np.log(y_pred),
                   np.log(1 - y_pred)).mean(axis=0)


def fit_log_reg(X, y, w_init=None, num_epochs=100, learning_rate=0.001, alpha=50, verbose=False):
    """Fits logistic regression model to given data using gradient descent
    Assumes that loss being used is Negative Log Likelihood with L2 regularization

    Arguments:
        X {ndarray} -- (num_elements, num_features) shaped ndarray of data
        y {ndarray} -- (num_elements, 1) shaped ndarray of binary (0/1) labels corresponding to data

    Keyword Arguments:
        num_epochs {int} -- number of iterations in gradient descent (default: {100})
        learning_rate {float} -- step size in gradient descent (default: {0.001})
        alpha {int} -- coefficient of l2 regularization (aka lambda) (default: {50})
        verbose {bool} -- prints loss and misc info during training, if set to True (default: {False})

    Returns:
        w {ndarray} -- (1, num_features+1) shaped ndarray of coefficients for log. reg.
    """
    # dataset meta data
    num_elements, num_features = X.shape

    # initialize w to small random values or if provided, some initial value
    if w_init is not None and w_init.shape == (num_elements, 1):
        w = w_init
    else:
        w = np.random.rand(1, num_features)

    for ne in range(num_epochs):
        # make predictions using current value of w
        y_pred = mysigmoid(X @ w.transpose())

        # calculate gradient of loss function
        nll_grad = (y_pred - y).transpose() @ X
        l2_grad = alpha * w
        nll_l2_grad =  nll_grad + l2_grad

        # print debug info every 250 steps, if in debug mode
        if verbose and (ne%250 == 0):
            print('nll_grad={:.5f} l2_grad={:.5f}'.format(np.abs(nll_grad).mean(), np.abs(l2_grad).mean()))
            print('ccr = {:.5f}'.format(((y_pred > 0.5) == y).mean()))

        # update w
        w -= learning_rate * nll_l2_grad

    return w

In [11]:
# fit logistic regression model
w = fit_log_reg(X_train, y_train, num_epochs=3000, learning_rate=0.00001, alpha=10, verbose=True)

nll_grad=7488.90245 l2_grad=4.99014
ccr = 0.39103
nll_grad=12873.73662 l2_grad=7.37099
ccr = 0.62609
nll_grad=7269.70466 l2_grad=10.56381
ccr = 0.50435
nll_grad=7381.62166 l2_grad=14.21048
ccr = 0.44891
nll_grad=752.41887 l2_grad=16.39328
ccr = 0.72663
nll_grad=6815.51639 l2_grad=18.83591
ccr = 0.61984
nll_grad=7324.88864 l2_grad=21.68851
ccr = 0.47255
nll_grad=7306.85055 l2_grad=23.76304
ccr = 0.48424
nll_grad=7168.60319 l2_grad=25.40845
ccr = 0.53641
nll_grad=11815.59797 l2_grad=27.25652
ccr = 0.64103
nll_grad=7270.45241 l2_grad=29.07273
ccr = 0.49946
nll_grad=8886.49261 l2_grad=30.12855
ccr = 0.67880


In [12]:
# predict on test data and train data
y_pred_test = mysigmoid(X_test @ w.transpose()) >= 0.5
y_pred_train = mysigmoid(X_train @ w.transpose()) >= 0.5

# calculate CCR
ccr_test = (y_pred_test == y_test).mean()
ccr_train = (y_pred_train == y_train).mean()
print('Test ccr = {:.5f}, Train ccr = {:.5f}'.format(ccr_test, ccr_train))

Test ccr = 0.67970, Train ccr = 0.69158
